### Bi-Metrics
#### Por Andrea Fuentes Pérez y Víctor Sánchez Auñón
#### 3º de Ingeniería Biomédica

VALIDACIÓN Y FORMATO DEL MRZ CON PRE-PROCESIONAMIENTO:

In [67]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor
from PIL import Image, ImageEnhance
import os

mrz = {
    "linea1": "",
    "linea2": "",
    "linea3": ""
}

model = ocr_predictor(pretrained=True)

def extract_text(ocr_model, image_file_path: str, mrz: bool = False, angulo: float = 0, algoritmo: Image.Resampling = Image.Resampling.NEAREST):
    """Extrae el texto de una imagen a partir de un modelo OCR"""
    
    # Abrimos, cropeamos, giramos y aumentamos para mejor lectura
    img = Image.open(image_file_path).convert('L')

    ancho, alto = img.size
    if mrz:
        img = img.crop((0, alto*0.6, ancho, alto))
        img = img.rotate(angulo, resample=algoritmo, expand=True)  
    img = ImageEnhance.Contrast(img).enhance(2.0)    
    temp_path = "temp_mrz_process.jpg"
    img.save(temp_path)

    doc = DocumentFile.from_images(temp_path)
    result = ocr_model(doc)
    
    # Limpiar archivo temporal
    if os.path.exists(temp_path):
        os.remove(temp_path)
        
    return result

def show_ocr_result(result):
    """Muestra una imagen de resultado y el texto extraido"""

    # mostrar resultado
    result.show()

    # mostrar texto con mas de 50% de confianza
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                words = []
                for word in line.words:
                    if word.confidence > 0.5:
                        words.append(word.value)
                
                
                print(' '.join(words).upper())

#Primera estraregia: Algoritmo BICUBIC, angulo de 0.5 para la línea 1 y 2, 0.3 para línea 3
def leerMRZ(path):
    text_back1 = extract_text(model, path, mrz=True, angulo=0.5, algoritmo=Image.Resampling.BICUBIC)
    text_back2 = extract_text(model, path, mrz=True, angulo=0.3, algoritmo=Image.Resampling.BICUBIC)

    mrz = {
        "linea1": text_back1.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back1.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back2.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia2MRZ(path)
    else:
        return mrz

#Segunda estrategia: Algoritmo NEAREST, angulo de 0.5 para las tres líneas
def estrategia2MRZ(path):
    text_back = extract_text(model, path, mrz=True, angulo=0.5, algoritmo=Image.Resampling.NEAREST)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta. Probando con el siguiente algoritmo...\n")
        estrategia3MRZ(path)
    else:
        return mrz

#Tercera estrategia: Algoritmo BILINEAL, angulo de 0.5 para las tres líneas
def estrategia3MRZ(path):
    text_back = extract_text(model, path, mrz=True, angulo=0.5, algoritmo=Image.Resampling.BILINEAR)

    mrz = {
        "linea1": text_back.pages[0].blocks[0].lines[0].words[0].value,
        "linea2": text_back.pages[0].blocks[0].lines[1].words[0].value,
        "linea3": text_back.pages[0].blocks[0].lines[2].words[0].value
    }

    mal = False
    for key, value in mrz.items():
        print(f"{key}: {value}")
        if len(value) != 30:
            mal = True

    if mal:
        print("\nLectura incorrecta, por favor repita la imagen\n")
    else:
        return mrz


#Obtenemos MRZ de una imagen
mrz = leerMRZ('./dni_fotos/ivan.jpg')

linea1: IDESPCCZ121138377039985J<<<<<<
linea2: 0508078M2709075ESP<<<<<<<<<<<3
linea3: GALIANO<SORO<<IVAN<<<<<<<<<<<<


CÁLCULO Y COMPROBACIÓN DE CHECKSUMS:

In [68]:
for key, value in mrz.items():
    print(f"{key}: {value}")

num_docu = mrz["linea1"][5:14]
fecha_nac = mrz["linea2"][0:6]
#control es L1 sin ID ni pais (5 al final) + L2 sin sexo ni nacionalidad (0 a 7 + 8 a 15 + 18 a 29)
control = mrz["linea1"][5:] + mrz["linea2"][0:7] + mrz["linea2"][8:15] + mrz["linea2"][18:29]
def calculoHash(campo):
    total = 0
    posi = 0 #empieza en 0 -> *7, posi++ 1 -> *3, posi++ 2 -> *1 y posi - 2 
    valoresHash = [7, 3, 1]
    for char in campo:
        total += valorCharHash(char) * valoresHash[posi]
        
        if posi == 2:
            posi = 0
        else:
            posi += 1
    return total % 10 # cheksum = resto de 10 (es el ultimo digito)

def valorCharHash(char):
    if char.isdigit():
        return int(char)
    elif char == "<":
        return 0
    else:
        # A=65, si tiene q valer 10, entonces ord(A) - 55 = 10
        return ord(char) - 55

print("Hash numero documento: ", calculoHash(num_docu)) #4
print("Hash fecha nacimiento: ", calculoHash(fecha_nac)) #8
print("Hash control final: ", calculoHash(control)) #7


linea1: IDESPCCZ121138377039985J<<<<<<
linea2: 0508078M2709075ESP<<<<<<<<<<<3
linea3: GALIANO<SORO<<IVAN<<<<<<<<<<<<
Hash numero documento:  3
Hash fecha nacimiento:  8
Hash control final:  3
